In [ ]:
# import statements for ARIMAX
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss


In [ ]:
# load data
df = pd.read_csv('AQI.csv')
df['time'] = pd.to_datetime(df['time'])

# set time as index
df.set_index('time', inplace=True)


In [ ]:
# function for stationarity test

def adf_test(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    print('\nADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])

def kpss_test(timeseries):
    result = kpss(timeseries, regression='c')
    print('\nKPSS Statistic: %f' % result[0])
    print('p-value: %f' % result[1])


In [ ]:
# visualize moving average
def plot_moving_average(timeseries, window):
    rolmean = timeseries.rolling(window=window).mean()
    rolstd = timeseries.rolling(window=window).std()
    plt.plot(timeseries, color='blue', label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

# plot moving average
plot_moving_average(df['PM2.5'], 24)


In [ ]:
variables = ['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR', 'PM2.5']

In [ ]:
# check for stationarity
for var in variables:
    print(var)
    adf_test(df[var])
    kpss_test(df[var])

In [ ]:
# differencing
for variable in variables:
    df[f'{variable}_diff'] = df[variable] - df[variable].shift(1)

In [ ]:
# removve first row
df = df.dropna()

In [ ]:
# check for stationarity
for var in variables:
    print(var)
    adf_test(df[f'{var}_diff'])
    kpss_test(df[f'{var}_diff'])

In [ ]:
# plot moving average of differenced data
plot_moving_average(df['PM2.5_diff'].dropna(), 24)


In [ ]:
# test for stationarity
for variable in variables:
    print(variable)
    adf_test(df[f'{variable}_diff'])
    print('\n')


In [ ]:
for variable in variables:
    print(variable)
    kpss_test(df[f'{variable}_diff'])
    print('\n')

In [ ]:
forecast_steps = 48

In [ ]:
# split data into 4 folds, each fold contains data from one season
df_winter_1 = df[(df.index.year == 2018) & (df.index.month < 3)]
df_spring_1 = df[(df.index.year == 2018) & (df.index.month >= 3) & (df.index.month < 6)]
df_summer_1 = df[(df.index.year == 2018) & (df.index.month >= 6) & (df.index.month < 9)]
df_fall_1 = df[(df.index.year == 2018) & (df.index.month >= 9) & (df.index.month < 12)]
df_winter_2 = df[((df.index.year == 2018) & (df.index.month >= 12)) | ((df.index.year == 2019) & (df.index.month < 3))]
df_spring_2 = df[(df.index.year == 2019) & (df.index.month >= 3) & (df.index.month < 6)]
df_summer_2 = df[(df.index.year == 2019) & (df.index.month >= 6) & (df.index.month < 9)]
df_fall_2 = df[(df.index.year == 2019) & (df.index.month >= 9) & (df.index.month < 12)]
df_winter_3 = df[((df.index.year == 2019) & (df.index.month >= 12)) | ((df.index.year == 2020) & (df.index.month < 3))]
df_spring_3 = df[(df.index.year == 2020) & (df.index.month >= 3) & (df.index.month < 6)]
df_summer_3 = df[(df.index.year == 2020) & (df.index.month >= 6) & (df.index.month < 9)]
df_fall_3 = df[(df.index.year == 2020) & (df.index.month >= 9) & (df.index.month < 12)]
df_winter_4 = df[((df.index.year == 2020) & (df.index.month >= 12)) | ((df.index.year == 2021) & (df.index.month < 3))]
df_spring_4 = df[(df.index.year == 2021) & (df.index.month >= 3) & (df.index.month < 6)]
df_summer_4 = df[(df.index.year == 2021) & (df.index.month >= 6) & (df.index.month < 9)]
df_fall_4 = df[(df.index.year == 2021) & (df.index.month >= 9) & (df.index.month < 12)]




df_list = [df_winter_1, df_spring_1, df_summer_1, df_fall_1, df_winter_2, df_spring_2, df_summer_2, df_fall_2, df_winter_3, df_spring_3, df_summer_3, df_fall_3, df_winter_4, df_spring_4, df_summer_4, df_fall_4]
diff_list = [f'{variable}_diff' for variable in variables]

# split train and validation data
def split_data(df_list, i, forecast_steps):
    train = df_list[i][:-168]
    valid = df_list[i][-168:]
    exog_train = train[diff_list]
    exog_valid = valid[diff_list]
    return train, valid, exog_train, exog_valid


In [ ]:
# ARIMAX model
def evaluate_arimax_model(y, X, order, forecast_steps):
    # Fit the model
    model = SARIMAX(y, exog=X, order = order, seasonal_order=(0,0,0,0))
    model_fit = model.fit(disp=False)
    aic = model_fit.aic
    # Make predictions
    forecast = model_fit.forecast(steps=forecast_steps, exog=X[-forecast_steps:])

    return forecast, aic

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

'''p_values = [1,2,3,4,5]
q_values = [0,1,2,3,4]'''

p_values = [1]
q_values = [0]

rmse_lst = []
mae_lst = []
aic_lst = []
best_score, best_cfg = float("inf"), None

for p in p_values:
    for q in q_values:
        order = (p, 0, q)
        rmse_lst.clear()  # Clear lists before each model order
        mae_lst.clear()
        aic_lst.clear()
        
        for fold in range(16):
            train, valid, exog_train, exog_valid = split_data(df_list, fold, forecast_steps)
            result, aic= evaluate_arimax_model(train['PM2.5_diff'], exog_train, order, forecast_steps)
            
            # Set index for result to match valid
            result.index = valid.index[:forecast_steps]
            rmse = np.sqrt(np.mean((result - valid['PM2.5_diff'])**2))
            mae = np.mean(np.abs(result - valid['PM2.5_diff']))
            
            rmse_lst.append(rmse)
            mae_lst.append(mae)
            aic_lst.append(aic)
            
            print('fold:', fold, 'RMSE:', rmse, 'MAE:', mae, 'AIC:', aic)
        
        # Calculate average RMSE, MAE, and AIC for validation
        rmse_mean = np.mean(rmse_lst)
        mae_mean = np.mean(mae_lst)
        aic_mean = np.mean(aic_lst)
        
        print('ARIMA%s RMSE=%.3f, MAE=%.3f, AIC=%.3f' % (order, rmse_mean, mae_mean, aic_mean))
        
        # If current AIC is lower than best_score, update best_cfg and best_score
        if aic_mean < best_score:
            best_score, best_cfg = aic_mean, order
            RMSE_valid, MAE_valid = rmse_mean, mae_mean

print('Best ARIMA%s AIC=%.3f' % (best_cfg, best_score))


In [ ]:
print('MAE_valid:', MAE_valid)
print('RMSE_valid:', RMSE_valid)

In [ ]:
p, d, q = 1, 0, 0

new_train = df
test = df[35064:]

# Train initial SARIMAX model
model = SARIMAX(new_train['PM2.5_diff'], exog=new_train[diff_list], order=(p, d, q))
model_fit = model.fit()


In [ ]:
# clear df index
new_df = df.reset_index(drop=True, inplace=False)
#re index from 0
new_df.index = range(len(new_df))

In [ ]:
# clear df index
def re_index(df_list):
    for df in df_list:
        df.reset_index(drop=True, inplace=True)
        df.index = range(len(df))

re_index(df_list)

In [ ]:
forecast_steps = 48 

# Initialize forecast_results DataFrame
forecast_results = pd.DataFrame(index=test.index, columns=[f'T+{i}' for i in range(1, forecast_steps+1)])
print(forecast_results.shape)

# Define starting point for rolling forecast
initial_data_points = -168


In [ ]:
# Perform rolling forecast
for idx in range(initial_data_points, len(new_df) - forecast_steps):
    # Prepare training data up to the current point
    rolling_train = new_df.loc[:new_df.index[idx - 1], 'PM2.5_diff']
    rolling_exog = new_df.loc[:new_df.index[idx - 1], diff_list]
    
    # Prepare exogenous variables for the forecast period
    current_exog = new_df.loc[new_df.index[idx:idx + forecast_steps], diff_list]
    
    # Forecast for the next 48 hours
    model = SARIMAX(rolling_train, exog=rolling_exog, order=(p, d, q))
    model_fit = model.fit(disp=False)
    forecast = model_fit.forecast(steps=forecast_steps, exog=current_exog)
    
    # Store forecast results
    forecast_results.loc[new_df.index[idx], :] = forecast.values

    # Optionally, add the actual observed value to the training data for the next iteration
    if idx + forecast_steps < len(new_df):
        observed_value = new_df.loc[new_df.index[idx + forecast_steps], 'PM2.5_diff']
        new_df.loc[new_df.index[idx + forecast_steps], 'PM2.5_diff'] = observed_value

# Print or use forecast_results
print(forecast_results.head())